##### 1. Setup Libraries

In [ ]:
# updated version at C:\Users\User\AppData\Roaming\MobaXterm\slash\RemoteFiles\264144_2_1

In [1]:
import pandas as pd
import numpy as np
import zipfile, Levenshtein, pycountry, re, json
from textdistance import jaro_winkler
from tqdm import tqdm

### 2. Importing and Elaborating Inputs

##### Extract Firms

In [2]:
# Import pBook Tab 
path = r'C:\Users\User\Dropbox\My PC (HP-16340)\Desktop\Work\Dropbox\Patent name match\comlist_forpatent.xlsx' 
cols = ['UID', 'CompanyName', 'Country/Region']
df_firms = pd.read_excel(path, usecols=cols)

#### Shorten Firms Country/Region
Note on Missing Values Refill for Firms Country/Region: 
- Firm Bygg had state California, but Country/Region empty, it was manually filled in the excel sheet as United States; 
- CELL AGRITECH SDN BHD had Country/Region empty, a quick google search revealts they are headquartered in Malaysia; 
- Health Sources Nutrition Co.,Ltd. had no Coutry/Region, but its headquarters are reported to be in Heifei, i.e. Mainland China. 

These changes were made directly in the source table i.e. pBook 

In [3]:
 
country_name_to_code = {country.name: country.alpha_2 for country in pycountry.countries}
mapping_missing_countries = {'Czech Republic' : 'CZ',
                        'India ' : 'IN',
                        'Mainland China' : 'CN',
                        'Russia' : 'RU',
                        'Scotland' : 'GB',
                        'South Korea' : 'KR',
                        'Taiwan' : 'TW',
                        'Venezuela' : 'VE',
                        'Vietnam' : 'VN',
                        'latvia' : 'LV'}
                        
country_name_to_code.update(mapping_missing_countries)
df_firms['Country/Region'] = df_firms['Country/Region'].map(country_name_to_code)
df_firms

,UID,CompanyName,Country/Region
0,657135,100Foods,BR
1,769415,108Labs,US
2,938559,1Ness Foods Pvt. Ltd.,IN
3,326285,3D Bio-Tissues Ltd.,GB
4,832680,4U Free From,ES
...,...,...,...
1471,978656,Zero Meat,JP
1472,551962,ZhenMeat,CN
1473,833239,Zikooin Company,KR
1474,653912,Zoglo's (brand of Soglowek),CA


##### Extract People

In [4]:
# Extract People
path_people1 = r'C:\Users\User\Dropbox\My PC (HP-16340)\Desktop\Work\Dropbox\Patent name match\Patstat2021a\data_PATSTAT_Global_2021_Autumn\tls206_part01.zip'
csv_people1 = 'tls206_part01.csv'
path_people2 = r'C:\Users\User\Dropbox\My PC (HP-16340)\Desktop\Work\Dropbox\Patent name match\Patstat2021a\data_PATSTAT_Global_2021_Autumn\tls206_part02.zip'
csv_people2 = 'tls206_part02.csv'

cols_people = ['person_id', 'person_name', 'person_ctry_code', 'psn_sector']

with zipfile.ZipFile(path_people1, 'r') as z:
    with z.open(csv_people1) as f:
            d_people1 = pd.read_csv(f, nrows = None, usecols=cols_people)
        
with zipfile.ZipFile(path_people2, 'r') as z:
    with z.open(csv_people2) as f:
            d_people2 = pd.read_csv(f, nrows = None, usecols=cols_people)

# concatenate 
d_people = pd.concat([d_people1, d_people2], axis=0)

#### Filter People for Applicants

In [5]:
# This is a better method to obtain a list of applicants
df_applicants = d_people.loc[d_people['psn_sector']=='COMPANY', d_people.columns!='psn_sector']
df_applicants.rename(columns={'person_id':'applicant_id', 'person_name':'applicant_name', 'person_ctry_code':'applicant_ctry_code'}, inplace=True)
df_applicants.reset_index(inplace=True, drop=True)
del d_people, d_people1, d_people2
df_applicants

,applicant_id,applicant_name,applicant_ctry_code
0,1,Nokia Corporation,FI
1,6,NOKIA MOBILE PHONES LTD.,FI
2,8,MEDIMMUNE LIMITED,GB
3,15,CAMBRIDGE ANTIBODY TECHNOLOGY LIMITED,GB
4,17,Cambridge Antibody Technology Limited,GB
...,...,...,...
9981848,79307363,"VERTEX TRADING CO., LTD.",WS
9981849,79307369,DETNET SOUTH AFRICA (PROPRIETARY) LIMITED,ZA
9981850,79307373,GLENCORE OPERATION SOUTH AFRICA (PROPRIETARY) ...,ZA
9981851,79307380,PowerOptimal(Pty) Ltd,ZA


#### 3. Missing-Management and Standardizations of Inputs

#### Signal Applicants Country Code in PATSTAT
Recall that there is no need to harmonize country code in PATSTAT. 
We could have to refill person name as it sometimes is missing, and we could have rather than nan coutry code, empyty coutry code in the shape of '  '

In [6]:
# Check how many applicants have empty coutry code
n = df_applicants['applicant_ctry_code'].isna().sum()
k = df_applicants.loc[df_applicants['applicant_ctry_code'] == "  ", :].count()[0]
print(f'there are {df_applicants.shape[0]} applicants and {n} of them have the coutry code missing')
print(f'there are {df_applicants.shape[0]} applicants and {k} of them have the coutry code empty')

there are 9981853 applicants and 97 of them have the coutry code missing
there are 9981853 applicants and 5212468 of them have the coutry code empty


In [7]:
# Whenever the coutry code is missing, replace it with XX
df_applicants.loc[df_applicants['applicant_ctry_code']=='  ', 'applicant_ctry_code'] = 'XX'

##### Standardize Companies and Firms Names

In [8]:
# Load the standardization dictionary from the JSON file
with open("standardization_dict.json", "r") as json_file:
    standardization_dict = json.load(json_file)

In [9]:
def standardize_names(name):

    try:
        name = re.sub(r'[^\w\s]', '', name.upper()) #leave only numbers and lettersn turn characters upper
        for key, val in standardization_dict.items(): #loop over legal designations and their corresponding WOS structure
            key, val = key.strip(), val.strip()
            name = name.replace(key, val)
        return name.strip()  # Strip any leading/trailing spaces
    except:
        return name

In [10]:
s = 'Edison, Gas UNIVERSITIES 15/12 Av.'
standardize_names(s)

'EDISON GAS UNIV 1512 AV'

In [11]:
# Clear names in both databases
df_firms['Clear Name Firm'] = (df_firms['CompanyName'].apply(standardize_names)).copy(deep=True)
df_applicants['Clear Name Applicant'] = (df_applicants['applicant_name'].apply(standardize_names)).copy(deep=True)

### 4. Fuzzy Matching of pBook Firms and Patstat Applicant Company Names

In [12]:
# Use alphanumeric matching on names without legal designation
df_perfect = pd.merge(df_applicants, df_firms, how='inner', left_on='Clear Name Applicant', right_on='Clear Name Firm')
df_perfect['Match Type'] = 'Alphanumeric'
df_perfect['Score'] = np.nan
df_perfect

,applicant_id,applicant_name,applicant_ctry_code,Clear Name Applicant,UID,CompanyName,Country/Region,Clear Name Firm,Match Type,Score
0,431494,Valio Ltd.,FI,VALIO LTD,896158,Valio LTD,FI,VALIO LTD,Alphanumeric,NaN
1,1287001,Valio Ltd.,FI,VALIO LTD,896158,Valio LTD,FI,VALIO LTD,Alphanumeric,NaN
2,3265298,VALIO LTD.,FI,VALIO LTD,896158,Valio LTD,FI,VALIO LTD,Alphanumeric,NaN
3,3392149,Valio Ltd.,FI,VALIO LTD,896158,Valio LTD,FI,VALIO LTD,Alphanumeric,NaN
4,3392153,Valio Ltd.,FI,VALIO LTD,896158,Valio LTD,FI,VALIO LTD,Alphanumeric,NaN
...,...,...,...,...,...,...,...,...,...,...
208,77477663,"Wild Earth, Inc",US,WILD EARTH INC,358287,Wild Earth Inc,US,WILD EARTH INC,Alphanumeric,NaN
209,74518409,DAIZ Inc.,JP,DAIZ INC,610404,"DAIZ, Inc.",JP,DAIZ INC,Alphanumeric,NaN
210,77359949,DAIZ INC.,JP,DAIZ INC,610404,"DAIZ, Inc.",JP,DAIZ INC,Alphanumeric,NaN
211,74844987,The Better Meat Co.,US,THE BETTER MEAT CO,493241,The Better Meat Co.,US,THE BETTER MEAT CO,Alphanumeric,NaN


In [13]:
# Fully Broadcasted JW Distance Implementation

# Set threshold for the Jaro Winkler score of two names. If their score exceeds the threshold, they are considered equal. 
threshold = 0.935
# vectorize the pyfunction get_jaro_distance so that it can take as input arrays of the same shape. 
vectorized_JW = np.vectorize(jaro_winkler)

# Create the ndarrays on which to apply vectorized_JW
firms_names = df_firms['Clear Name Firm'].values
applicants_names = df_applicants['Clear Name Applicant'].values
applicants_names = np.reshape(applicants_names, (-1, 1))
broadcasted_firms_names, broadcasted_applicants_names = np.broadcast_arrays(firms_names, applicants_names)
# Apply get_jaro_distance on each couple of names from firms on the horizontal axis, and applicants on the vertical axis. 
# Save in JW whether the result exceeded the threshold or not. 
JW = vectorized_JW(broadcasted_firms_names, broadcasted_applicants_names)

# Place the firms index as columns of the resulting boolean dataframe, and the applicant index as its index. Thus create a DataFrame.
# Then stack it and reset the index as to obtain a form [applicant_index][firm_index][Jaro-Winkler score of the names at those positions]
JW = pd.DataFrame(JW, index=df_applicants['Clear Name Applicant'].index, columns=df_firms['Clear Name Firm'].index)
JW = JW.stack()
JW = JW.reset_index()
JW.columns = ['applicant_index', 'firm_index', 'JW_score']

# Retreive the couples of applicant and firm indeces which scored above the threshold, store them in filtering masks. Then, apply these marsks on the original
# firm and applicant dataframes to recover DataFrames with full rows the names of which exceeded the threshold. Concatenate them,  insert Match Type = Jaro-Winkler,
# and insert the JW score of the corresponding names
index_mask = JW.loc[(JW['JW_score']>threshold), ['applicant_index', 'firm_index']] 
applicants_to_append = (df_applicants.loc[index_mask['applicant_index']]).reset_index(drop=True)
firms_to_append = (df_firms.loc[index_mask['firm_index']]).reset_index(drop=True)
df_JW = pd.concat([applicants_to_append, firms_to_append], axis=1)
df_JW['Match Type'] = 'Jaro-Winkler'
df_JW['Score'] = JW.loc[index_mask.index, 'JW_score'].values

df_JW

MemoryError: Unable to allocate 110. GiB for an array with shape (9981853, 1476) and data type object

In [ ]:
# Fully Broadcasted LV Distance Implementation

# Set threshold for the Levenstein score of two names, which is the max number of character changes between two strings for them to be considered equal
threshold = 2
# vectorize the pyfunction get_jaro_distance so that it can take as input arrays of the same shape. 
vectorized_LV = np.vectorize(Levenshtein.distance)

# Create the arrays on which to apply vectorized_LV
firms_names = df_firms['Clear Name Firm'].values
applicants_names = df_applicants['Clear Name Applicant'].values
applicants_names = np.reshape(applicants_names, (-1, 1))
broadcasted_firms_names, broadcasted_applicants_names = np.broadcast_arrays(firms_names, applicants_names)
# Apply Levenstein distance on each couple of names from firms on the horizontal axis, and applicants on the vertical axis. 
# Save in LV whether the result is below the threshold or not. 
LV = vectorized_LV(broadcasted_firms_names, broadcasted_applicants_names)

# Place the firms index as columns of the resulting boolean dataframe, and the applicant index as its index. Thus create a DataFrame.
# Then stack it and reset the index as to obtain a form [applicant_index][firm_index][boolean of whether the names at those positions are below the threshold]
LV = pd.DataFrame(LV, index=df_applicants['Clear Name Applicant'].index, columns=df_firms['Clear Name Firm'].index)
LV = LV.stack()
LV = LV.reset_index()
LV.columns = ['applicant_index', 'firm_index', 'LV_score']

# Retreive the couples of applicant and firm indeces which scored true, store them in filtering masks. Then, apply these marsks on the original
# firm and applicant dataframes to recover DataFrames with full rows the names of which exceeded the threshold. Concatenate them and insert Match Type = Levenshtein. 
index_mask = LV.loc[(LV['LV_score']<threshold), ['applicant_index', 'firm_index']] 
applicants_to_append = (df_applicants.loc[index_mask['applicant_index']]).reset_index(drop=True)
firms_to_append = (df_firms.loc[index_mask['firm_index']]).reset_index(drop=True)
df_LV = pd.concat([applicants_to_append, firms_to_append], axis=1)
df_LV['Match Type'] = 'Levenshtein'
df_LV['Score'] = LV.loc[index_mask.index, 'LV_score'].values

df_LV

,applicant_id,applicant_name,applicant_ctry_code,Clear Name Applicant,UID,CompanyName,Country/Region,Clear Name Firm,Match Type,Score
0,17312,VALEO,FR,VALEO,357006,Paleo,BE,PALEO,Levenshtein,1
1,17406,MEURA,BE,MEURA,586864,Heura,ES,HEURA,Levenshtein,1


In [ ]:
# Merge the resulting tables togerther
df_fuzzy = pd.concat([df_perfect, df_JW, df_LV], axis = 0)
df_fuzzy = df_fuzzy.loc[:, (df_fuzzy.columns != 'Clear Name Applicant')&(df_fuzzy.columns != 'Clear Name Firm')]
df_fuzzy.reset_index(drop=True, inplace=True)
df_fuzzy.drop_duplicates(inplace=True)
df_fuzzy

,applicant_id,applicant_name,applicant_ctry_code,UID,CompanyName,Country/Region,Match Type,Score
0,50051403,Impossible Foods Inc.,US,765140,Impossible Foods,US,Jaro-Winkler,0.96
1,17312,VALEO,FR,357006,Paleo,BE,Levenshtein,1.00
2,17406,MEURA,BE,586864,Heura,ES,Levenshtein,1.00


### 5. Additional Filters and Disambiguations
Use country code to disambiguate between rows with double matches

#### Generate Quality of Location Match

In [ ]:
def match_quality(applicant_ctry, firm_ctry):
    if (applicant_ctry == 'XX') or (firm_ctry == 'XX'):
        return 'Inconclusive'
    elif (applicant_ctry == firm_ctry):
        return 'Matched'
    elif (applicant_ctry != firm_ctry):
        return 'Not Matched'

In [ ]:
df_fuzzy['Location Match'] = df_fuzzy.apply(lambda row: match_quality(row['applicant_ctry_code'], row['Country/Region']), axis=1)
df_fuzzy

,applicant_id,applicant_name,applicant_ctry_code,UID,CompanyName,Country/Region,Match Type,Score,Location Match
0,50051403,Impossible Foods Inc.,US,765140,Impossible Foods,US,Jaro-Winkler,0.96,Matched
1,17312,VALEO,FR,357006,Paleo,BE,Levenshtein,1.00,Not Matched
2,17406,MEURA,BE,586864,Heura,ES,Levenshtein,1.00,Not Matched


#### Disambiguate on Goodness of Name and Location Matches

In [ ]:

# create optimality metric as the cartesian distance of the match (Type, Location) score from (3, 3) which is the maximum attainable.
d_loc = {'Not Matched': 1, 'Inconclusive':2, 'Matched':3}
d_type = {'Levenshtein':1, 'Jaro-Winkler':2, 'Alphanumeric':3}
disamb_df_fuzzy = df_fuzzy.copy()
disamb_df_fuzzy['opt_distance'] = np.sqrt((disamb_df_fuzzy['Location Match'].map(d_loc))**2 + (disamb_df_fuzzy['Match Type'].map(d_type))**2)

# Ensure that one company maps exactly on one applicant, then that one applicant maps exactly on one comapny. 
# Note that idmin() will return the index of the first occurrence of the minimum value, with no tie handling
for col in tqdm(['UID', 'applicant_id']):
    idx_min_rows = disamb_df_fuzzy.groupby(col)['opt_distance'].idxmin()
    disamb_df_fuzzy = disamb_df_fuzzy.loc[idx_min_rows]

# Display the result
disamb_df_fuzzy.drop('opt_distance', axis=1, inplace=True)
disamb_df_fuzzy

100%|██████████| 2/2 [00:00<00:00, 497.43it/s]


,applicant_id,applicant_name,applicant_ctry_code,UID,CompanyName,Country/Region,Match Type,Score,Location Match
1,17312,VALEO,FR,357006,Paleo,BE,Levenshtein,1.00,Not Matched
2,17406,MEURA,BE,586864,Heura,ES,Levenshtein,1.00,Not Matched
0,50051403,Impossible Foods Inc.,US,765140,Impossible Foods,US,Jaro-Winkler,0.96,Matched


### 6. Opportunities for Further Elaborations
Use the threshold to filter, then the country code, then filter on industry information. 
Look at IPC class in patstat, it's not industry but they are activities related to the industry, so that you know food companies would only patent in certain classes 
and biotech companies would only patent in those other classes. 
Identify IPC classes in which food & beverage firms would make a patent, but the name and the courty code is good enough for now. 

### 7. Final Files Production

For the final output:

The output should be a table with CrunchBase company ID, name, country, year of fundation and PATSTAT applicant ID, name, address country. Then it should include the type of match, and the filtering criterion.


1. Add a column to the excel file where you identify whether the firm has a patent or not. Write type of match, score of match, whether coutry and region was correct. 
2. For those with a patent, the output should look like the sample patent shared with me - patent number, applicant ID etc etc. that's the real final file. 
3. Make the four output files in the examples. 
4. write what you did and why for a person who does not know much about programming. This can be done after hadning in the files.  

#### Add Columns to pBook File

In [ ]:
# Re-import pBook File
path = r'C:\Users\User\Dropbox\My PC (HP-16340)\Desktop\Work\Dropbox\Patent name match\comlist_forpatent.xlsx' 
df_firms = pd.read_excel(path)
df_firms

# Perform a Left Join between pBook and disamb_df_fuzzy on UID
right_merge = disamb_df_fuzzy.loc[:, ['UID', 'applicant_id', 'Match Type', 'Score', 'Location Match']]
df_paste = pd.merge(df_firms, right_merge, on='UID', how='left')
df_paste.rename(columns={'applicant_id':'Matched'}, inplace=True)
df_paste['Matched'] = (~df_paste['Matched'].isna())
df_paste

,UID,CompanyName,Brand Name,Brief Description,Country/Region,State,City,Website,Year Founded,Matched,Match Type,Score,Location Match
0,657135,100Foods,NaN,Brazil-based company that produces plant-based...,Brazil,NaN,São Paulo,https://www.100foods.com.br/,2018.0,False,NaN,NaN,NaN
1,769415,108Labs,NaN,US-based company creating cultivated human milk,United States,California,Davis,https://108labs.net/,2013.0,False,NaN,NaN,NaN
2,938559,1Ness Foods Pvt. Ltd.,NaN,India-based producer of plant-based dairy alte...,India,NaN,Mumbai,https://1ness.in/,2019.0,False,NaN,NaN,NaN
3,326285,3D Bio-Tissues Ltd.,NaN,U.K.-based research entity aimed at improving ...,United Kingdom,NaN,Newcastle,https://www.3dbiotissues.com/,2019.0,False,NaN,NaN,NaN
4,832680,4U Free From,NaN,Spain-based company producing a variety of veg...,Spain,NaN,Alicante,https://4ufreefrom.com/en/,2019.0,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471,978656,Zero Meat,NaN,Japan-based company producing soy meat products,Japan,NaN,NaN,https://zeromeat.jp/,2018.0,False,NaN,NaN,NaN
1472,551962,ZhenMeat,NaN,China-based company producing domestic plant-b...,Mainland China,NaN,Beijing,https://zhenmeat.com/en,2017.0,False,NaN,NaN,NaN
1473,833239,Zikooin Company,NaN,South Korea-based company producing plant-base...,South Korea,NaN,Seoul,www.unlimeat.com,2017.0,False,NaN,NaN,NaN
1474,653912,Zoglo's (brand of Soglowek),NaN,"Produces frozen soy-based meats, including hot...",Canada,ON,Toronto,http://www.zoglos.com/,1987.0,False,NaN,NaN,NaN


In [ ]:
# write resulting dataframe to the output sheet of the pBook excel
path = r'C:\Users\User\Dropbox\My PC (HP-16340)\Desktop\Work\Dropbox\Patent name match\Process Code - Michele\comlist_forpatent result version.xlsx'
sheet_name = 'Output Sheet'  # Replace with the desired sheet name

# Write the DataFrame to the specified sheet
with pd.ExcelWriter(path, engine='openpyxl', mode='w') as writer:
    df_paste.to_excel(writer, sheet_name=sheet_name, index=False)

#### Produce Output pbookappln_ipc
This table must have as headers the ID of an application, the symbol of its IPC class, and the count of how often that symbol appears in the database. All this data is found in  tls209

In [ ]:
# Extract Internatonal Patent Classification Tab
path_class1 = r'C:\Users\User\Dropbox\My PC (HP-16340)\Desktop\Work\Dropbox\Patent name match\Patstat2021a\data_PATSTAT_Global_2021_Autumn\tls209_part01.zip'
csv_class1 = 'tls209_part01.csv'
path_class2 = r'C:\Users\User\Dropbox\My PC (HP-16340)\Desktop\Work\Dropbox\Patent name match\Patstat2021a\data_PATSTAT_Global_2021_Autumn\tls209_part02.zip'
csv_class2 = 'tls209_part02.csv'

cols_people = ['appln_id', 'ipc_class_symbol']

with zipfile.ZipFile(path_class1, 'r') as z:
    with z.open(csv_class1) as f:
            d_class1 = pd.read_csv(f, nrows=None, usecols=cols_people)
        
with zipfile.ZipFile(path_class2, 'r') as z:
    with z.open(csv_class2) as f:
            d_class2 = pd.read_csv(f, nrows=None, usecols=cols_people)

# concatenate 
d_class = pd.concat([d_class1, d_class2], axis=0)
del d_class1, d_class2

In [ ]:
# implement the procedure over the entire table
pbookappln_ipc = d_class.groupby('ipc_class_symbol').count()
pbookappln_ipc.rename(columns={'appln_id' : 'count'}, inplace=True)
pbookappln_ipc = pd.merge(d_class, pbookappln_ipc, how='left', left_on='ipc_class_symbol', right_index=True)

# Specify the desktop folder and save the DataFrame to a CSV file
desktop_folder = r'C:\Users\User\Dropbox\My PC (HP-16340)\Desktop\Work\Dropbox\Patent name match\Process Code - Michele'  
csv_file_path = f'{desktop_folder}/pbookapplnipc.csv'
pbookappln_ipc.to_csv(csv_file_path, index=False)
pbookappln_ipc.head()

,appln_id,ipc_class_symbol,IPC_macro_class_x,count,IPC_macro_class_y
0,1,G06K 7/00,6,35,35
1,1,G06K 17/00,6,98,98
2,1,G06K 19/077,6,48,48
3,1,H01R 12/18,1,3,3
4,1,H04M 1/02,4,34,34


#### Produce Output pbookappln_ipc35
The second and third position in the IPC_CLASS_SYMBOL string represent the application macro class from 01 to 99. I will retreive application ID, specify which IPC macro class they belong to - if it is withing the first 35. Then, outline the fraction of applicaitons in that macroclass with respect to all the applicaitons. 

In [ ]:
# Implement the procedure over the entire database
wdf = d_class.copy(deep=True)
wdf['IPC_macro_class'] = wdf['ipc_class_symbol'].apply(lambda s: int(s[1:3]))
wdf = wdf.groupby('IPC_macro_class').count()/wdf.shape[0]
wdf.rename(columns={'appln_id':'frequency'}, inplace=True)
wdf.reset_index(inplace=True)
wdf = wdf.loc[wdf['IPC_macro_class']<=35, ['IPC_macro_class', 'frequency']]

pbookappln_ipc35 = d_class
pbookappln_ipc35['IPC_macro_class'] = pbookappln_ipc35['ipc_class_symbol'].apply(lambda s: int(s[1:3]))
pbookappln_ipc35 = pd.merge(pbookappln_ipc35, wdf, how='left', on='IPC_macro_class')
pbookappln_ipc35 = pbookappln_ipc35.loc[:, pbookappln_ipc35.columns!='ipc_class_symbol']
pbookappln_ipc35.drop_duplicates(inplace=True) #here I am dropping duplicates,
# if fact, one appln_ID could belong to more IPC classes, each mapping on the same Macro Class, with analogous frequence. Thus creating redundant records.

# Specify the desktop folder and save the DataFrame to a CSV file
desktop_folder = r'C:\Users\User\Dropbox\My PC (HP-16340)\Desktop\Work\Dropbox\Patent name match\Process Code - Michele'  
csv_file_path = f'{desktop_folder}/pbookapplnipc35.csv'
pbookappln_ipc35.to_csv(csv_file_path, index=False)
pbookappln_ipc35.head()

,appln_id,IPC_macro_class,frequency
0,1,6,0.06270
3,1,1,0.16903
4,1,4,0.13187
8,2,7,0.05609
18,2,12,0.02794


#### Produce Output pbookcompanypat
This table has the Company ID, Appln_ID, Family_ID, Earliest_Filing_Year, Filing Office, Count_References, Count_Citing, Grant_Year

In [ ]:
# Extract Applications
path_application1 = r'C:\Users\User\Dropbox\My PC (HP-16340)\Desktop\Work\Dropbox\Patent name match\Patstat2021a\data_PATSTAT_Global_2021_Autumn\tls201_part01.zip'
csv_application1 = 'tls201_part01.csv'
path_application2 = r'C:\Users\User\Dropbox\My PC (HP-16340)\Desktop\Work\Dropbox\Patent name match\Patstat2021a\data_PATSTAT_Global_2021_Autumn\tls201_part02.zip'
csv_application2 = 'tls201_part02.csv'
path_application3 = r'C:\Users\User\Dropbox\My PC (HP-16340)\Desktop\Work\Dropbox\Patent name match\Patstat2021a\data_PATSTAT_Global_2021_Autumn\tls201_part03.zip'
csv_application3 = 'tls201_part03.csv'

cols_application = ['appln_id', 'docdb_family_id', 'inpadoc_family_id', 'earliest_filing_year', 'receiving_office']

with zipfile.ZipFile(path_application1, 'r') as z:
    with z.open(csv_application1) as f:
            d_application1 = pd.read_csv(f, nrows = None, usecols=cols_application)

with zipfile.ZipFile(path_application2, 'r') as z:
    with z.open(csv_application2) as f:
            d_application2 = pd.read_csv(f, nrows = None, usecols=cols_application)

with zipfile.ZipFile(path_application3, 'r') as z:
    with z.open(csv_application3) as f:
            d_application3 = pd.read_csv(f, nrows = 500, usecols=cols_application)

# concatenate results
d_application = pd.concat([d_application1, d_application2, d_application3], axis=0)
del d_application1, d_application2, d_application3
d_application.head()

,appln_id,receiving_office,earliest_filing_year,docdb_family_id,inpadoc_family_id
0,0,,9999,0,0
1,1,,1999,8554171,1
2,2,,1991,27517085,2
3,3,,1999,7915918,3
4,4,,1999,22889365,4


In [ ]:
# Extract the Application-Person Link
path_link = r'C:\Users\User\Dropbox\My PC (HP-16340)\Desktop\Work\Dropbox\Patent name match\Patstat2021a\data_PATSTAT_Global_2021_Autumn\tls207_part01.zip'
csv_link = 'tls207_part01.csv'
cols_link = ['person_id', 'appln_id']

with zipfile.ZipFile(path_link, 'r') as z:
    with z.open(csv_link) as f:
            d_link = pd.read_csv(f, nrows = None, usecols=cols_link)
d_link.head()

,person_id,appln_id
0,1,1
1,1,7
2,1,46
3,1,775
4,1,1192


In [ ]:
pbookcompanypat = pd.merge(disamb_df_fuzzy[['UID', 'applicant_id']], d_link, left_on='applicant_id', right_on='person_id', how='inner').drop('person_id', axis=1)
pbookcompanypat = pd.merge(pbookcompanypat, d_application, on='appln_id', how='inner')

# Specify the desktop folder and save the DataFrame to a CSV file
desktop_folder = r'C:\Users\User\Dropbox\My PC (HP-16340)\Desktop\Work\Dropbox\Patent name match\Process Code - Michele'  
csv_file_path = f'{desktop_folder}/pbookcompanypat.csv'
pbookcompanypat.to_csv(csv_file_path, index=False)
pbookcompanypat.head()

,UID,applicant_id,appln_id,receiving_office,earliest_filing_year,docdb_family_id,inpadoc_family_id
0,297418,1,1,,1999,8554171,1
1,297418,1,7,,1999,10861834,7
2,297418,1,46,,2002,9938265,46


#### Produce Output pbookmatchfinal
This table has Company ID, year founded, (year closed), (pbookcy), pbookname, patstat name, type match, (person_ID, patent cy, has patents, type match2). 
I will use this table to return my results in their entirety. 

In [ ]:
right_merge = disamb_df_fuzzy.loc[:, ['UID', 'applicant_id', 'applicant_name', 'applicant_ctry_code', 'Match Type', 'Score', 'Location Match']]
left_merge = df_firms.loc[:, ['UID', 'CompanyName', 'Country/Region', 'Website', 'Year Founded']]
pbookmatchfinal = pd.merge(left_merge, right_merge, on='UID', how='inner')

# Specify the desktop folder and save the DataFrame to a CSV file
desktop_folder = r'C:\Users\User\Dropbox\My PC (HP-16340)\Desktop\Work\Dropbox\Patent name match\Process Code - Michele'  
csv_file_path = f'{desktop_folder}/pbookmatchfinal.csv'
pbookmatchfinal.to_csv(csv_file_path, index=False)
pbookmatchfinal.head()

,UID,CompanyName,Country/Region,Website,Year Founded,applicant_id,applicant_name,applicant_ctry_code,Match Type,Score,Location Match
0,431107,Alt Co. (The Alt Company),India,https://alt.company/,2020.0,380,Alstom Technology Ltd,CH,Jaro-Winkler,0.801569,Not Matched
1,897234,Australia's Own,Australia,https://australiasownfoods.com.au/,2017.0,486,Austriamicrosystems AG,AT,Jaro-Winkler,0.820504,Not Matched
2,297418,Kinda Co,United Kingdom,https://www.thekindaco.com/,2017.0,1,Nokia Corporation,FI,Jaro-Winkler,0.810714,Not Matched
3,499722,"Luyef Biotechnologies, Inc.",Chile,https://luyef.com,2020.0,26,Lucent Technologies Inc.,US,Jaro-Winkler,0.801569,Not Matched
4,184200,Marin Food,Japan,https://www.marinfood.co.jp/english/,1957.0,46,Marioff Corporation Oy,FI,Jaro-Winkler,0.841667,Not Matched
